In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import os.path
import requests
import json

In [4]:
# in case is not the right path
try:
    os.chdir(os.path.join(os.getcwd(), 'notebooks')) 
    print(os.getcwd())
except:
    pass

# Data cleaning

## Processing `paper_references.csv`

In [2]:
papers_df = pd.read_csv("../mappings/data/raw-paper_references.csv")

Null values in the DOI column are represented as different strings, so we remove all values that do not look like a DOI (do not start witn "10.")

In [6]:
papers_df['DOI'] = papers_df['DOI'].replace('^(?!10).*', '', regex=True)
papers_df.head(5)

,ID,valid,No_de_Ref,new_ref,filename,DOI,Title,Year,Journal,Pages,Issue,Abstract,Authors,Volume,Corresponding_author,Country,Country_name
0,1,1,1,1,1-s2.0-S092633731000086X-main,10.1016/j.apcatb.2010.02.030,"""Effect of silver doping on the TiO2 for photo...",2010,APPLIED CATALYSIS B-ENVIRONMENTAL,239,NaN,"""Pure TiO2 and various silver-enriched TiO2 po...","""K. Ko\u010d\u00ed, K. Mat\u011bj\u016f, L. Ob...",96,kamila.koci@vsb.cz,"""CZ,CZ,CZ,CZ,CZ,CZ,CZ,CZ,CZ""","""Czech Republic,Czech Republic,Czech Republic,..."
1,2,1,2,2,10.1016@j.cattod.2009.07.067,10.1016/j.cattod.2009.07.067,"""CO2 reforming into fuel using TiO2 photocatal...",2009,CATALYSIS TODAY,341,NaN,"""It was previously reported that CO2 could be ...","""A. Nishimura, N. Komatsu, G. Mitsui, M. Hirot...",148,nisimura@mach.mie-u.ac.jp,"""JP,JP,JP,JP,AU""","""Japan,Japan,Japan,Japan,Australia"""
2,3,1,3,3,1-s2.0-S0926860X05006125-main,10.1016/j.apcata.2005.08.021,"""Photo reduction of CO2 to methanol using opti...",2005,APPLIED CATALYSIS A-GENERAL,194,NaN,"""Greenhouse gases such as CO2 are the primary ...","""J.C.S. Wu, H-M. Lin ,C-L. Lai""",269,cswu@ntu.edu.tw,"""TW,TW,TW""","""Taiwan,Taiwan,Taiwan"""
3,4,1,4,4,27362718_Photo_reduction_of_CO2_to_methanol_vi...,10.1155/S1110662X05000176,"""Photo reduction of CO2 to methanol via TiO2 p...",2005,INTERNATIONAL JOURNAL OF PHOTOENERGY,115,NaN,"""Greenhouse gas such as CO2 is the primary cau...","""J.C.S. Wu, H-M. Lin ,C-L. Lai""",7,cswu@ntu.edu.tw,"""TW,TW""","""Taiwan,Taiwan"""
4,5,1,5,5,10.1023@A@1011403320301,10.1023/A:1011403320301,"""A new type of photocatalysis initiated by pho...",2000,CATALYSIS SURVEYS FROM JAPAN,107,2,"""ZrO2 has been found to be an effective photoc...","""S. Yoshida, Y. Kohno""",4,artleafs.admin@artleafs.eu,"""JP,JP""","""Japan,Japan"""


### Export

In [19]:
papers_df.to_csv("../mappings/data/paper_references.csv", index=False, sep=",")

## OpenAlex papers

### Taking list of DOIs

In [7]:
papers_df = pd.read_csv("../mappings/data/paper_references_curated.csv", sep=';')

Manual corrections in erroneous DOIs in `paper_references_curated.csv`
- 10.22146/ijc.841 --> 10.22146/ijc.21248
- paper con doi (incorrecto) 10.1007/s11244-017-0797 is repeated, the correct DOI is 10.1007/s11244-017-0797-x (appears in a previous row)
- 10.6084/m9.figshare.12715484 --> 10.1038/s41467-020-18350-7
- 10.1002/anie.x0xx00000x --> 10.1002/ange.202012019
- 10.1007/s10562-020-03426 --> 10.1007/s10562-020-03426-2
- 10.1002/slct.20200 --> 10.1002/slct.202004839

Papers without DOI:
- Kinetic study of photocatalytic reduction of CO2 over TiO2 --> https://openalex.org/W1629652031
- A study on the photoreduction of green house CO2 gas catalized by TiO2 to form methane an methanol --> not in OpenAlex
- Selective ethanol synthesis from carbon dioxide --> 10.1595/003214097x414166170
- Selective photoreduction of CO2HCO3 - to formate by aqueous suspensions and colloids of Pd-TiO2 --> 10.1021/j100372a080
 


### Downloading individual JSON files per paper from the DOI

In [ ]:
headers = {'Accept': 'application/json'}
query ='https://api.openalex.org/works/https://doi.org/{}'

#for doi in papers_df['DOI']:
for index, row in papers_df.iterrows():
    response = requests.get(query.format(row['DOI']), headers=headers)
    
    if response.status_code == 200:
        res_json = response.json()
        res_json['solar_id'] = row['No_de_Ref']
        with open('data/jsonOA/'+str(row['No_de_Ref'])+'.json', 'w') as file:
            file.write(json.dumps(res_json, indent=4))
    else:
        print(f"Error with {row['DOI']}, id {str(row['No_de_Ref'])}")
    


Error with nan, id 28
Error with nan, id 145
Error with 10.1007/s11244-017-0797, id 519


### Merging individual JSONs into one single file

In [ ]:
path = '../mappings/data/jsonOA/'
json_file_names = os.listdir(path)
json_file_names.remove('.DS_Store')

In [ ]:
merged_json = []
for file in json_file_names:
    filename = path + file
    with open(filename, 'r') as infile:
        merged_json.append(json.load(infile))
    
with open('../mappings/data/papersOA.json', 'w') as out_json:
    json.dump(merged_json, out_json)

## Processing `catalystdata.csv`

In [7]:
exp_df = pd.read_csv("../mappings/data/raw-catalystsdata.csv")

### Cleaning null values
Negative numbers and values that serve as null

In [8]:
exp_df.replace([-1,"-1", -1.0, 0.0, 9999.99], '', inplace=True)
exp_df.head(5)

,ID,No_de_Ref,Catalyst_name,TiO2_crystal_structure,Catalyst,Support,support_percent,Co_catalyst,co_catalyst_2,co_catalyst_3,...,C4H10_mol_m2h,C5H10_mol_m2h,C5H12_mol_m2h,CH3OH_mol_m2h,C2H5OH_mol_m2h,CH3COH_mol_m2h,HCOOH_mol_m2h,CH2O_mol_m2h,C2H4O2_mol_m2h,timestamp
0,1,1,TiO2,Anatase,TiO2,NaN,,NaN,NaN,NaN,...,,,,0.01,,,,,,0000-00-00 00:00:00
1,2,1,TiO2,Anatase,TiO2,NaN,,Ag,NaN,NaN,...,,,,0.01,,,,,,0000-00-00 00:00:00
2,3,1,Ag/TiO2,Anatase,TiO2,NaN,,Ag,NaN,NaN,...,,,,0.01,,,,,,0000-00-00 00:00:00
3,4,1,Ag/TiO2,Anatase,TiO2,NaN,,Ag,NaN,NaN,...,,,,0.02,,,,,,0000-00-00 00:00:00
4,5,1,Ag/TiO2,Anatase,TiO2,NaN,,Ag,NaN,NaN,...,,,,0.03,,,,,,0000-00-00 00:00:00


### Cleaning values in columns  with numeric values 
In the column `Masscat_g` some values are float (g), others have different measurements corresponding to different concepts or even comments. Filtering out all that do not correspond to grams.

Similar issues happen for the percentages of other types of inputs for co-catalysts, supports etc. 

In [9]:
def clean_numeric_colums(value):
    try:
        value = float(value)
        if value == 0.0:
            return('')
        else:
            return(value)
    
    except ValueError or TypeError:
        return('')
    
exp_df.Masscat_g = exp_df.Masscat_g.apply(clean_numeric_colums)
exp_df.support_percent = exp_df.support_percent.apply(clean_numeric_colums)
exp_df.percent = exp_df.percent.apply(clean_numeric_colums)
exp_df.percent_cc_2 = exp_df.percent_cc_2.apply(clean_numeric_colums)
exp_df.percent_cc_3 = exp_df.percent_cc_3.apply(clean_numeric_colums)
exp_df.dopant_percent = exp_df.dopant_percent.apply(clean_numeric_colums)
exp_df.dye_percent = exp_df.dye_percent.apply(clean_numeric_colums)

### Aligning values with ontology hierarchies

#### Reactor types

In [10]:
exp_df['Reactor_type'] = exp_df['Reactor_type'].replace(('\s', 'Batch'), '', regex=True)
exp_df.Reactor_type.unique()

array(['Slurry', 'Fixed-bed', 'OpticalFiber', 'Monolithic', 'Membrane',
       'Fluidised-bed', ''], dtype=object)

#### Light Sources

In [ ]:
exp_df['Light_source'] = exp_df['Light_source'].replace('\s', '', regex=True)
exp_df['Light_source'] = exp_df['Light_source'].replace({"UV":"https://purl.obolibrary.org/obo/OBI_0002900", 
                              "Visible":"https://purl.obolibrary.org/obo/OBI_0002901", 
                              "Vis":"https://purl.obolibrary.org/obo/OBI_0002901", 
                              "Solar":"https://purl.obolibrary.org/obo/OBI_0002902",  
                              "UV-Vis":"https://w3id.org/solar/o/core#UltravioletVisibleLightSource", 
                              "SolarSimulator":"https://w3id.org/solar/o/core#SolarSimulatorLightSource", 
                              "Monochromatic":"https://w3id.org/solar/o/core#MonochromaticLightSource"})
exp_df.Light_source.unique()

array(['https://purl.obolibrary.org/obo/OBI_0002900',
       'https://purl.obolibrary.org/obo/OBI_0002902',
       'https://purl.obolibrary.org/obo/OBI_0002901',
       'https://w3id.org/solar/o/core#UltravioletVisibleLightSource',
       'https://w3id.org/solar/o/core#SolarSimulatorLightSource',
       'https://w3id.org/solar/o/core#MonochromaticLightSource'],
      dtype=object)

#### Lamps

In [16]:
exp_df['Lamp'] = exp_df['Lamp'].replace({"Mercury(Hg)":"https://w3id.org/solar/o/core#MercuryLamp", 
                              "Xenon(Xe)":"https://w3id.org/nfdi4cat/voc4cat_0000168", 
                              "Solar":"https://w3id.org/solar/o/core#SolarSimulatorLamp",  
                              "Not spedified":"", 
                              "Mercury-Xenon(Hg-Xe)":"https://w3id.org/nfdi4cat/voc4cat_0000169", 
                              "Tungsten(W)":"https://w3id.org/solar/o/core#TungstenLamp", 
                              "Fluorescent":"https://w3id.org/solar/o/core#FluorescentLamp", 
                              "Halogen":"https://w3id.org/solar/o/core#HalogenLamp", 
                              "Tungsten-Halide":"https://w3id.org/solar/o/core#Tungsten-HalideLamp", 
                              "LED":"https://w3id.org/solar/o/core#LEDLamp", 
                              "Other":"https://w3id.org/solar/o/core#OtherLamp"})
exp_df.Lamp.unique()

array(['https://w3id.org/solar/o/core#MercuryLamp',
       'https://w3id.org/solar/o/core#FluorescentLamp',
       'https://w3id.org/nfdi4cat/voc4cat_0000168',
       'https://w3id.org/solar/o/core#SolarSimulatorLamp',
       'https://w3id.org/solar/o/core#HalogenLamp', '',
       'https://w3id.org/solar/o/core#Tungsten-HalideLamp',
       'https://w3id.org/nfdi4cat/voc4cat_0000169',
       'https://w3id.org/solar/o/core#OtherLamp',
       'https://w3id.org/solar/o/core#LEDLamp',
       'https://w3id.org/solar/o/core#TungstenLamp', nan], dtype=object)

#### Operation modes

In [18]:
exp_df['Operation_mode'] = exp_df['Operation_mode'].replace({ 
                              "Batch":"https://w3id.org/nfdi4cat/voc4cat_0000110",
                              "Continuous":"https://w3id.org/nfdi4cat/voc4cat_0000109",})
exp_df.Operation_mode.unique()

array(['https://w3id.org/nfdi4cat/voc4cat_0000110',
       'https://w3id.org/nfdi4cat/voc4cat_0000109'], dtype=object)

#### Wavelengths

In [19]:
def set_wavelengths(wl):
    try:
        wl = float(wl)
        if (wl) >= 192 and (wl) <= 280:
            return("192-280")
        elif (wl) >= 280 and (wl) <= 315:
            return("280-315")
        elif (wl) >= 315 and (wl) <= 400:
            return("315-400")
        elif (wl) >= 400 and (wl) <= 780:
            return("400-780")
        elif (wl) >= 315 and (wl) <= 780:
            return("315-780")
        elif (wl) >= 280 and (wl) <= 780:
            return("280-780")
        elif (wl) >= 192 and (wl) <= 780:
            return("192-780")
        
    except ValueError:
        wl = wl.replace('192-280(UV-A)', '192-280')
        wl = wl.replace('315-400(UV-C)', '315-400')
        wl = wl.replace('280-315(UV-B)', '280-315')
        return(wl)
    
    except TypeError:
        return('')
    

exp_df.Wavelength_nm = exp_df.Wavelength_nm.apply(set_wavelengths)
exp_df.Wavelength_nm.unique()    


array(['192-280', '315-400', '280-315', '400-780', None, '315-780',
       '192-780', '280-780', ''], dtype=object)

### CO2 production

In [20]:
def co2_production(CO, CH4, C2H4, C2H6, C3H6, C3H8, C4H8, C4H10, C5H10, C5H12, CH3OH, C2H5OH, CH3COH, HCOOH, CH2O, C2H4O2):
    values = []
    for var in [CO, CH4, CH3OH, HCOOH, CH2O]:
        try:
            float(var)
            values.append(var)
        except:
            pass

    for var in [C2H4, C2H6, C2H5OH, C2H4O2, CH3COH]:
        try:
            float(var)
            values.append(var*2)
        except:
            pass

    for var in [C3H6, C3H8]:
        try:
            float(var)
            values.append(var*3)
        except:
            pass

    for var in [C4H8, C4H10]:
        try:
            float(var)
            values.append(var*4)
        except:
            pass

    for var in [C5H10, C5H12]:
        try:
            float(var)
            values.append(var*5)
        except:
            pass
    if sum(values) == 0.0:
        return('')
    else:
        return(sum(values))

In [21]:
#print(co2_production(exp_df.CO_mol_gh.iloc[0], exp_df.CH4_mol_gh.iloc[0], exp_df.C2H4_mol_gh.iloc[0], exp_df.C2H6_mol_gh.iloc[0], exp_df.C3H6_mol_gh.iloc[0], exp_df.C3H8_mol_gh.iloc[0], exp_df.C4H8_mol_gh.iloc[0], exp_df.C4H10_mol_gh.iloc[0], exp_df.C5H10_mol_gh.iloc[0], exp_df.C5H12_mol_gh.iloc[0], exp_df.CH3OH_mol_gh.iloc[0], exp_df.C2H5OH_mol_gh.iloc[0], exp_df.CH3COH_mol_gh.iloc[0], exp_df.HCOOH_mol_gh.iloc[0], exp_df.CH2O_mol_gh.iloc[0], exp_df.C2H4O2_mol_gh))

exp_df['co2_prod_molgh'] = exp_df.apply(lambda x: co2_production(x['CO_mol_gh'], x['CH4_mol_gh'], x['C2H4_mol_gh'], x['C2H6_mol_gh'], x['C3H6_mol_gh'], x['C3H8_mol_gh'], x['C4H8_mol_gh'], x['C4H10_mol_gh'], x['C5H10_mol_gh'], x['C5H12_mol_gh'], x['CH3OH_mol_gh'], x['C2H5OH_mol_gh'], x['CH3COH_mol_gh'], x['HCOOH_mol_gh'], x['CH2O_mol_gh'], x['C2H4O2_mol_gh']), axis=1)
exp_df['co2_prod_molg'] = exp_df.apply(lambda x: co2_production(x['CO_mol_g'], x['CH4_mol_g'], x['C2H4_mol_g'], x['C2H6_mol_g'], x['C3H6_mol_g'], x['C3H8_mol_g'], x['C4H8_mol_g'], x['C4H10_mol_g'], x['C5H10_mol_g'], x['C5H12_mol_g'], x['CH3OH_mol_g'], x['C2H5OH_mol_g'], x['CH3COH_mol_g'], x['HCOOH_mol_g'], x['CH2O_mol_g'], x['C2H4O2_mol_g']), axis=1)
exp_df['co2_prod_molm2h'] = exp_df.apply(lambda x: co2_production(x['CO_mol_m2h'], x['CH4_mol_m2h'], x['C2H4_mol_m2h'], x['C2H6_mol_m2h'], x['C3H6_mol_m2h'], x['C3H8_mol_m2h'], x['C4H8_mol_m2h'], x['C4H10_mol_m2h'], x['C5H10_mol_m2h'], x['C5H12_mol_m2h'], x['CH3OH_mol_m2h'], x['C2H5OH_mol_m2h'], x['CH3COH_mol_m2h'], x['HCOOH_mol_m2h'], x['CH2O_mol_m2h'], x['C2H4O2_mol_m2h']), axis=1)

In [ ]:
exp_df['co2_prod_molg']


### Catalyst selectivity 


In [22]:
def selectivity(product, H2, CO, CH4, CH3OH):
    if product == '':
        return('')
    
    values = []
    for value in [H2, CO, CH4, CH3OH]:
        try:
            values.append(float(value))
        except:
            values.append(0.0)

    sum_values = sum(values)
    return ((product/sum_values)*100)
    

In [23]:
exp_df['h2_selectivity_molg'] = exp_df.apply(lambda x: selectivity( x['H2_mol_g'], x['CO_mol_g'], x['CH4_mol_g'], x['H2_mol_g'], x['CH3OH_mol_g']), axis=1)
exp_df['ch4_selectivity_molg'] = exp_df.apply(lambda x: selectivity( x['CH4_mol_g'], x['CO_mol_g'], x['CH4_mol_g'], x['H2_mol_g'], x['CH3OH_mol_g']), axis=1)
exp_df['co_selectivity_molg'] = exp_df.apply(lambda x: selectivity( x['CO_mol_g'], x['CO_mol_g'], x['CH4_mol_g'], x['H2_mol_g'], x['CH3OH_mol_g']), axis=1)
exp_df['ch3oh_selectivity_molg'] = exp_df.apply(lambda x: selectivity( x['CH3OH_mol_g'], x['CO_mol_g'], x['CH4_mol_g'], x['H2_mol_g'], x['CH3OH_mol_g']), axis=1)


### Export

In [24]:
exp_df.to_csv("../mappings/data/catalystsdata.csv", index=False, sep=",")